In [1]:
%pip install torch torchvision torchaudio scikit-image matplotlib

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/78/18/7a2e56e2dc45a433dea9e1bf46a65e234294c9c470ccb4d4b53025f57b23/torch-2.5.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/b2/1b/b8eb51f87626c125cfa81f07488ab277e68e1c021c6cf2750d779eb61358/torchvision-0.20.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/09/2f/62c8925ffdcac92ea64f00ccc77e6262f3212a0cc22c21473a9fd98966cd/torchaudio-2.5.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scikit-image from https://files.pythonhosted.org/packages/50/b2/d5e97115733e2dc657e99868ae0237705b79d0c81f6ced21b8f0799a30d1/scikit_image-0.24.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/8b/ce/15b0bb2fb29b3d46211d8ca740b96b5232499fc49200b


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary tools (libraries) to work with data, images, and neural networks
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from skimage import io
from skimage.transform import resize
import torch.utils.data as data


In [3]:
%pip install python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
data_directory = random = os.getenv("DATASET1")
all_files = np.array(os.listdir(data_directory))
all_files = [f for f in all_files if f.endswith('.png') and f[-5] in ['0', '1']]


if os.path.exists(data_directory):
    print("Directory exists:", data_directory)
    print(os.listdir(data_directory))
else:
    print("Directory does not exist:", data_directory)


Directory exists: C:\Users\tejbu\Tejas\IIT Patna\Sem7\TB_Project\dataset\ChinaSet_AllFiles\ChinaSet_AllFiles\CXR_png
['CHNCXR_0001_0.png', 'CHNCXR_0002_0.png', 'CHNCXR_0003_0.png', 'CHNCXR_0004_0.png', 'CHNCXR_0005_0.png', 'CHNCXR_0006_0.png', 'CHNCXR_0007_0.png', 'CHNCXR_0008_0.png', 'CHNCXR_0009_0.png', 'CHNCXR_0010_0.png', 'CHNCXR_0011_0.png', 'CHNCXR_0012_0.png', 'CHNCXR_0013_0.png', 'CHNCXR_0014_0.png', 'CHNCXR_0015_0.png', 'CHNCXR_0016_0.png', 'CHNCXR_0017_0.png', 'CHNCXR_0018_0.png', 'CHNCXR_0019_0.png', 'CHNCXR_0020_0.png', 'CHNCXR_0021_0.png', 'CHNCXR_0022_0.png', 'CHNCXR_0023_0.png', 'CHNCXR_0024_0.png', 'CHNCXR_0025_0.png', 'CHNCXR_0026_0.png', 'CHNCXR_0027_0.png', 'CHNCXR_0028_0.png', 'CHNCXR_0029_0.png', 'CHNCXR_0030_0.png', 'CHNCXR_0031_0.png', 'CHNCXR_0032_0.png', 'CHNCXR_0033_0.png', 'CHNCXR_0034_0.png', 'CHNCXR_0035_0.png', 'CHNCXR_0036_0.png', 'CHNCXR_0037_0.png', 'CHNCXR_0038_0.png', 'CHNCXR_0039_0.png', 'CHNCXR_0040_0.png', 'CHNCXR_0041_0.png', 'CHNCXR_0042_0.png', 

In [14]:
# Define how we want to prepare our images for the neural network
# This converts the image to a format the network can understand
data_transforms = transforms.Compose([transforms.ToTensor()])


# This function loads and prepares each image for our neural network
def CNNloader(data_root, filename):
    # Load the image and resize it to 224x224 pixels
    filename_actual = data_root + '/' + filename
    data_old = io.imread(filename_actual)
    data_old = resize(data_old, (224, 224))
    data_old = np.array(data_old, dtype=np.float32)
    
    # If the image is black and white, convert it to a color image
    # (Our neural network expects color images)
    if len(data_old.shape) <= 2:
        data_new = np.zeros(data_old.shape + (3,))
        data_new[:,:,0] = data_new[:,:,1] = data_new[:,:,2] = np.array(data_old)
        data_old = np.array(data_new, dtype=np.float32)
    
    # Apply our predefined transformations
    data_old = data_transforms(np.array(data_old))
    return data_old

# This class helps organize our data and labels
class CNNDataLayer(data.Dataset):
    def __init__(self, data_root, filenames, loader):
        self.data_root = data_root
        self.filenames = filenames
        self.loader = loader

    # This tells the computer how to get each image and its label
    def __getitem__(self, index):
        filename = self.filenames[index]
        # We assume the label (0 or 1) is the second-to-last character in the filename
        target = [int(filename[-5])]
        target = torch.from_numpy(np.array(target))
        data = self.loader(self.data_root, filename)
        return data, target

    # This tells the computer how many images we have
    def __len__(self):
        return len(self.filenames)



In [15]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

device = None
if torch.backends.mps.is_available():
    device = torch.device('mps')  # Use Metal Performance Shaders on macOS
else:
    device = torch.device('cpu')  # Fallback to CPU

# print("Using device:", device)
# Get a list of all our image files and shuffle them randomly
all_files = np.array(os.listdir(data_directory))
np.random.shuffle(all_files)

# Split our data: 70% for training, 30% for testing
# train_files = all_files[:round(0.7*all_files.shape[0])]
train_files = all_files[:round(0.70*all_files.shape[0])]
test_files = all_files[round(0.70*all_files.shape[0]):]
print(train_files.shape, test_files.shape)

# Create our datasets using the custom class we defined earlier
data_sets_train = CNNDataLayer(data_root=data_directory, filenames=train_files, loader=CNNloader)
data_sets_test = CNNDataLayer(data_root=data_directory, filenames=test_files, loader=CNNloader)

# Create data loaders that will help feed data to our neural network in batches
data_loaders_train = data.DataLoader(data_sets_train, batch_size=8, shuffle=True, num_workers=0)
data_loaders_test = data.DataLoader(data_sets_test, batch_size=8, shuffle=True, num_workers=0)

# Load a pre-trained neural network (VGG16) that's good at understanding images
model_to_train = models.vgg16(pretrained=True)
# model_to_train.load_state_dict(torch.load("./vgg16-397923af.pth"))

# We don't want to retrain the whole network, so we "freeze" most of it
for param in model_to_train.features.parameters():
    param.require_grad = False

# Modify the last part of the network to focus on our specific task (finding tuberculosis)
num_features = model_to_train.classifier[6].in_features
features = list(model_to_train.classifier.children())[:-1]
features.extend([nn.Linear(num_features, 1), nn.Sigmoid()])
model_to_train.classifier = nn.Sequential(*features)

# Alternatively, we could use a custom-made neural network
# from own_cnn_model import SimpleCNN
# model_to_train = SimpleCNN()
model_to_train = model_to_train.to(device)

# Set up the "loss function" (how the network measures its mistakes)
# and the "optimizer" (how the network learns from its mistakes)
criterion = nn.BCELoss()
optimizer_ft = optim.SGD(model_to_train.parameters(), lr=0.001, momentum=0.9)

# Start the training process
num_epochs = 5  # We'll train for 100 rounds
for epoch in range(1, num_epochs+1):
    print('Epoch is ' + str(epoch))
    train_loss = 0
    test_loss = 0
    total_misclassified_train = total_count_train = 0
    total_misclassified_test = total_count_test = 0
    
    # Training phase
    model_to_train.train()
    for batch_idx, (data_now, target_now) in enumerate(data_loaders_train):
        # Move data to GPU if available
        data_now, target_now = data_now.to(device), target_now.to(device)
        
        # Make predictions
        target_output_model = model_to_train(data_now)
        target_now = target_now.type(torch.FloatTensor).to(device)
        
        # Calculate loss (how wrong the predictions were)
        target_loss = criterion(target_output_model, target_now)
        
        # Calculate how many predictions were wrong
        misclassified_temp = target_output_model[target_now == 1]
        total_misclassified_train += (misclassified_temp < 0.5).sum().item()
        misclassified_temp = target_output_model[target_now == 0]
        total_misclassified_train += (misclassified_temp >= 0.5).sum().item()
        total_count_train += data_now.shape[0]
        
        # Learn from the mistakes (backpropagation)
        optimizer_ft.zero_grad()
        target_loss.backward()
        optimizer_ft.step()
    
    # Testing phase (similar to training, but we don't learn from these)
    model_to_train.eval()

    # Store all predictions and true labels for metrics
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_idx, (data_now, target_now) in enumerate(data_loaders_test):
            data_now, target_now = data_now.to(device), target_now.to(device)
            target_output_model = model_to_train(data_now)
            
            # Convert outputs to binary predictions
            predicted_labels = (target_output_model >= 0.5).float()
            all_predictions.extend(predicted_labels.cpu().numpy())
            all_targets.extend(target_now.cpu().numpy())
            
            # Calculate how many predictions were wrong
            misclassified_temp = target_output_model[target_now == 1]
            total_misclassified_test += (misclassified_temp < 0.5).sum().item()
            misclassified_temp = target_output_model[target_now == 0]
            total_misclassified_test += (misclassified_temp >= 0.5).sum().item()
            total_count_test += data_now.shape[0]
    
    # Convert lists to numpy arrays for metric calculations
    all_predictions = np.array(all_predictions).flatten()
    all_targets = np.array(all_targets).flatten()
    
    # Calculate evaluation metrics
    accuracy = np.mean(all_predictions == all_targets)
    precision = precision_score(all_targets, all_predictions)
    recall = recall_score(all_targets, all_predictions)
    f1 = f1_score(all_targets, all_predictions)
    
    # Print evaluation metrics
    print(f'Epoch {epoch} - Test Metrics:')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    
    # Save our progress (the network's current state)
    snapshot_path = './snapshots_trial'
    os.makedirs(snapshot_path, exist_ok=True)
    snapshot_name = f'epoch-{epoch}-trainerror-{total_misclassified_train/total_count_train:.4f}-testerror-{total_misclassified_test/total_count_test:.4f}.pth'
    torch.save(model_to_train.state_dict(), os.path.join(snapshot_path, snapshot_name))



(463,) (199,)
Epoch is 1
Epoch 1 - Test Metrics:
Accuracy: 0.7588
Precision: 0.9362
Recall: 0.4944
F1 Score: 0.6471
Epoch is 2
Epoch 2 - Test Metrics:
Accuracy: 0.8392
Precision: 0.9014
Recall: 0.7191
F1 Score: 0.8000
Epoch is 3
Epoch 3 - Test Metrics:
Accuracy: 0.8392
Precision: 0.8904
Recall: 0.7303
F1 Score: 0.8025
Epoch is 4
Epoch 4 - Test Metrics:
Accuracy: 0.8392
Precision: 0.8904
Recall: 0.7303
F1 Score: 0.8025
Epoch is 5
Epoch 5 - Test Metrics:
Accuracy: 0.8693
Precision: 0.9200
Recall: 0.7753
F1 Score: 0.8415


In [27]:
# Example of how to load a saved model for later use
model_loaded = torch.load("./snapshots/epoch-5-trainerror-0.2857-testerror-0.4286.pth", map_location=lambda storage, loc: storage)
model_to_train.load_state_dict(model_loaded)

/var/folders/bt/f868ty6d66g2scxrj62lpgzc0000gn/T/ipykernel_88117/3568264980.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loaded = torch.load("./snapshots/epoch-

FileNotFoundError: [Errno 2] No such file or directory: './snapshots/epoch-5-trainerror-0.1447-testerror-0.1558.pth'